# Python &rarr; OKL Sandbox

AST Grammar info can be found in the [ast docs](https://docs.python.org/3/library/ast.html#abstract-grammar)

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from typing import List
import ast
import inspect

from occa import okl
from occa.okl import Const, Exclusive, Shared
from occa.okl import Oklifier, py2okl
from occa.okl.attrorderer import AttrOrderer

In [28]:
def add(a: Const[float], 
        b: Const[float]) -> float:
    return a + b

@okl.kernel
def add_vectors(a: Const[List[float]],
                b: Const[List[float]],
                ab: List[float]) -> None:
    for i in range(3):
        pass
    for i in okl.range(len(a)).tile(16)(1)[2]:
        # Type tests
        foo: bool = True
        bar: np.float64 = 3.2
        s_foo: Shared[List[float, 20, 30]]
        e_foo: Exclusive[float]
        bar: Const[List[float, 2]] = [1,2]
            
        s: str = 'string'
            
        # Flow tests
        for j in range(10):
            pass
        
        if i > 10:
            continue
        elif i < 5:
            break
        else:
            i: int = 3
            return 3
        
        while True:
            pass
            
        # Closure tests
        ab[i] = add(a[i], b[i])

In [29]:
print(add_vectors.__okl_source__)

KeyError: 'value'

In [15]:
foo = okl.Oklifier.get_last_error_node()